In [ ]:
!pip install openai

     |████████████████████████████████| 42 kB 1.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 162 kB 6.8 MB/s 
  Created wheel for openai: filename=openai-0.18.1-py3-none-any.whl size=53168 sha256=f275ef03356eaf9e23545a547a3620e62f97ca086214dda3adec6b7795d2c765
  Stored in directory: /root/.cache/pip/wheels/5a/bf/24/fcdc9d2b81f9c7e565bb2036ec9f7cc930056b829895b3bf48
Successfully built openai


In [ ]:
import os
import openai
import copy
import scipy.stats
import sklearn.datasets
import pandas as pd
from scipy.io.arff import loadarff 
import urllib
import re

## Outliers

### Load data

#### Load outliers detected

In [ ]:
outliers_dict = {}
outliers_dict["wine"] = {"Alcohol": [], "Malic_acid": [5.8, 5.51, 5.65], "Ash": [3.22, 1.36, 3.23], "Alcalinity_of_ash": [10.6, 30, 28.5, 28.5], "Magnesium": [151, 139, 136, 162], "Total_phenols": [], "Flavanoids": [], "Nonflavanoid_phenols": [], "Proanthocyanins": [3.28, 3.58], "Color_intensity": [10.8, 13, 11.75], "Hue": [1.71], "OD280%2FOD315_of_diluted_wines": [], "Proline": []}
outliers_dict["ecoli"] = {"mcg": [], "gvh": [0.88, 0.84, 0.9, 0.84, 0.86, 0.86, 1, 0.86, 0.83, 0.86, 0.84, 0.84, 0.83], "lip": [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], "chg": [1], "aac": [0.16, 0, 0.06, 0.83, 0.86, 0.83, 0.88, 0.84, 0.17], "alm1": [], "alm2": []}
outliers_dict["glass"] = {"RI": [1.53393, 1.52475, 1.51115, 1.5232, 1.53125, 1.51215, 1.52777, 1.52365, 1.52315, 1.5241, 1.52725, 1.52369, 1.51131, 1.52614, 1.52664, 1.52739, 1.52667], "Na": [11.45, 17.38, 10.73, 15.79, 11.23, 11.02, 11.03], "Mg": [], "Al": [2.68, 0.34, 3.5, 2.74, 0.47, 0.51, 2.54, 2.42, 3.04, 2.34, 2.88, 0.29, 2.38, 3.02, 0.47, 2.51, 2.66, 2.79], "Si": [74.55, 70.16, 75.41, 69.89, 70.26, 69.81, 70.43, 70.48, 74.45, 70.57, 75.18, 70.7], "K": [1.68, 6.21, 1.46, 2.7, 1.76, 6.21, 1.41], "Ca": [16.19, 11.32, 13.24, 6.65, 5.87, 13.3, 11.41, 14.4, 11.14, 11.53, 5.79, 10.99, 10.79, 11.64, 11.27, 12.24, 5.43, 13.44, 14.68, 14.96, 6.47, 11.62, 11.22, 11.52, 12.5, 10.88], "Ba": [0.11, 0.61, 1.59, 0.14, 0.24, 1.57, 0.06, 0.15, 2.2, 0.09, 0.54, 0.81, 3.15, 1.38, 0.56, 1.55, 1.68, 1.59, 1.71, 0.66, 0.67, 1.57, 0.27, 1.63, 1.06, 1.64, 0.11, 0.69, 1.19, 0.63, 0.64, 0.53, 2.88, 0.76, 1.67, 0.64, 0.09, 0.4], "Fe": [0.28, 0.34, 0.51, 0.26, 0.32, 0.37, 0.28, 0.31, 0.3, 0.29, 0.28, 0.35]}
outliers_dict["iris"] = {"Sepal.Length": [], "Sepal.Width": [4.4, 4.1, 4.2, 2], "Petal.Length": [], "Petal.Width": []}
outliers_dict["Wisconsin-breast-cancer-cytology-features"] = {"thickness": [], "size": [], "shape": [], "adhesion": [10, 10, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 10, 10, 10, 10, 10, 10, 10], "single": [8, 8, 10, 8, 10, 8, 10, 8, 10, 8, 10, 10, 10, 10, 8, 10, 10, 8, 9, 10, 8, 8, 10, 8, 8, 10, 8, 10, 10, 10, 10, 8, 8, 10, 10, 10, 8, 10, 9, 10, 10, 10, 8, 10, 8, 8, 10, 10, 10, 10, 8, 10, 10, 8], "nuclei": [], "chromatin": [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10], "nucleoli": [10, 9, 10, 10, 9, 10, 9, 9, 9, 10, 10, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 10, 9, 10, 10, 9, 10, 10, 10, 10, 10, 10, 10, 9, 9, 10, 9, 10, 10, 10, 10, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10], "mitosis": [5, 4, 2, 4, 3, 2, 3, 2, 2, 5, 2, 7, 4, 10, 8, 10, 3, 7, 10, 3, 10, 4, 2, 8, 7, 3, 2, 3, 8, 3, 5, 2, 3, 3, 7, 3, 6, 2, 7, 3, 10, 7, 3, 10, 3, 2, 3, 5, 6, 4, 2, 2, 2, 2, 10, 4, 10, 2, 3, 3, 3, 3, 5, 3, 2, 8, 2, 8, 2, 3, 10, 4, 2, 10, 4, 4, 6, 2, 2, 3, 4, 3, 8, 3, 3, 10, 4, 7, 3, 4, 3, 8, 2, 2, 3, 2, 10, 5, 2, 7, 2, 2, 2, 2, 2, 2, 3, 2, 10, 3, 3, 10, 3, 2, 3, 7, 3, 8, 2, 2]}
outliers_dict["mpg"] = {"displ": [], "year": [], "cyl": [], "cty": [28, 28, 33, 35, 29], "hwy": [44, 44, 41]}
outliers_dict["ozone"] = {"V4": [33, 34, 33, 33, 38], "V5": [5480, 5410, 5350, 5480, 5490, 5470, 5320, 5420, 5440, 5480, 5450, 5500], "V6": [11, 11, 11, 11], "V7": [], "V8": [], "V9": [], "V10": [], "V11": [], "V12": [], "V13": [300, 350, 350, 300, 300, 300, 300, 300, 300, 300, 300, 300, 500, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300]}

In [ ]:
for dataset in outliers_dict:
  for attribute in outliers_dict[dataset]:
    outliers_dict[dataset][attribute] = list(set(outliers_dict[dataset][attribute]))

#### Load datasets

In [ ]:
datasets_dict = {}

In [ ]:
open_ml_datasets = [187, 39, 41, 43611, 41197]

for dataset in open_ml_datasets:
  try:
    dataset = sklearn.datasets.fetch_openml(data_id=dataset)
    datasets_dict[dataset["details"]["name"]] = dataset["data"]
  except:
    print(f"Dataset not available to fetch: {dataset}")

Dataset not available to fetch: 41197


In [ ]:
r_datasets = {"iris": "https://drive.google.com/u/0/uc?id=19txCu1LtIbPUeoJehlLGATXfvlEobsMH&export=download", "mpg": "https://drive.google.com/u/0/uc?id=1OeP9IVxVkhw7tiRNShiVfuhsGTFkjLEw&export=download"}

for elem in r_datasets:
  datasets_dict[elem] = pd.read_csv(r_datasets[elem])

In [ ]:
open_ml_from_file_dataset = {"ozone": "https://drive.google.com/u/0/uc?id=1gGQdl4T9mSY7lNEv5b3yPmkMK5oe9EUO&export=download"}

for elem in open_ml_from_file_dataset:
  urllib.request.urlretrieve(open_ml_from_file_dataset[elem], elem)
  datasets_dict[elem] = pd.DataFrame(loadarff(elem)[0])

#### Pre-process datasets

In [ ]:
datasets_dict["iris"].drop(["Species"], axis=1, inplace=True)
datasets_dict['Wisconsin-breast-cancer-cytology-features'].drop(["class", "id"], axis=1, inplace=True)
datasets_dict["mpg"].drop(["manufacturer", "model", "trans", "drv", "fl", "class"], axis=1, inplace=True)
datasets_dict["ozone"].drop(["V1", "V2", "V3"], axis=1, inplace=True)

In [ ]:
for dataset in datasets_dict:
  dataset_pd = copy.deepcopy(datasets_dict[dataset])
  datasets_dict[dataset] = {}
  for column in dataset_pd.columns:
    datasets_dict[dataset][column] = dataset_pd[column].dropna()

### Run experiment

In [ ]:
def generate_prompt(data):
  prompt = f'Calculate the outliers in "{data}"'
  return prompt

In [ ]:
openai.api_key = "COMPLETE"

In [ ]:
results = {}
n = 1
temp = 0

for dataset in datasets_dict:
  print(f"Dataset: {dataset}")
  results[dataset] = {}

  for attribute in datasets_dict[dataset]:
    print(f"Attribute: {attribute}")
    results[dataset][attribute] = {}

    prompt = generate_prompt(str(datasets_dict[dataset][attribute].to_list())[1:][:-1])

    for rep in range(0, n):

      response = openai.Completion.create(engine="text-davinci-002", prompt=prompt, temperature=temp, max_tokens=1000)
      text = response["choices"][0]["text"]
      print(prompt)
      print(text)

      results[dataset][attribute][rep] = text

Dataset: wine
Attribute: Alcohol
Calculate the outliers in "14.23, 13.2, 13.16, 14.37, 13.24, 14.2, 14.39, 14.06, 14.83, 13.86, 14.1, 14.12, 13.75, 14.75, 14.38, 13.63, 14.3, 13.83, 14.19, 13.64, 14.06, 12.93, 13.71, 12.85, 13.5, 13.05, 13.39, 13.3, 13.87, 14.02, 13.73, 13.58, 13.68, 13.76, 13.51, 13.48, 13.28, 13.05, 13.07, 14.22, 13.56, 13.41, 13.88, 13.24, 13.05, 14.21, 14.38, 13.9, 14.1, 13.94, 13.05, 13.83, 13.82, 13.77, 13.74, 13.56, 14.22, 13.29, 13.72, 12.37, 12.33, 12.64, 13.67, 12.37, 12.17, 12.37, 13.11, 12.37, 13.34, 12.21, 12.29, 13.86, 13.49, 12.99, 11.96, 11.66, 13.03, 11.84, 12.33, 12.7, 12.0, 12.72, 12.08, 13.05, 11.84, 12.67, 12.16, 11.65, 11.64, 12.08, 12.08, 12.0, 12.69, 12.29, 11.62, 12.47, 11.81, 12.29, 12.37, 12.29, 12.08, 12.6, 12.34, 11.82, 12.51, 12.42, 12.25, 12.72, 12.22, 11.61, 11.46, 12.52, 11.76, 11.41, 12.08, 11.03, 11.82, 12.42, 12.77, 12.0, 11.45, 11.56, 12.42, 13.05, 11.87, 12.07, 12.43, 11.79, 12.37, 12.04, 12.86, 12.88, 12.81, 12.7, 12.51, 12.6, 12.

In [ ]:
results

{'Wisconsin-breast-cancer-cytology-features': {'adhesion': {0: '\n\n1.0, 3.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.

## Evaluate

### Clean output

In [ ]:
non_decimal = re.compile(r'[^\d.,]+')

In [ ]:
results_clean = copy.deepcopy(results)
for dataset in results_clean:
  for attribute in results_clean[dataset]:
    for rep in results_clean[dataset][attribute]:
      results_clean[dataset][attribute][rep] = non_decimal.sub("", results_clean[dataset][attribute][rep]).split(",")
      if results_clean[dataset][attribute][rep] == ["."]:
        results_clean[dataset][attribute][rep] = []


In [ ]:
n = 1
evaluation = {}
latex_dict_list = []
for dataset in results_clean:
  latex_dict = {}
  print(f"Dataset: {dataset}\n")
  latex_dict["dataset"] = dataset
  latex_dict["experiment"] = "outliers"
  latex_dict["prompt"] = "prompt1"
  evaluation[dataset] = {}
  evaluation[dataset]["total_hits"] = 0

  for attribute in results_clean[dataset]:

    evaluation[dataset][attribute] = {}
    evaluation[dataset][attribute]["total_hits"] = 0

    for rep in results_clean[dataset][attribute]:

      if results_clean[dataset][attribute][rep] == [] and outliers_dict[dataset][attribute] != []:
        evaluation[dataset][attribute][f"hit_{rep}"] = 0
      if all(elem in outliers_dict[dataset][attribute] for elem in results_clean[dataset][attribute][rep]):
        evaluation[dataset][attribute][f"hit_{rep}"] = 1
        evaluation[dataset][attribute]["total_hits"] += 1
      elif any(elem in outliers_dict[dataset][attribute] for elem in results_clean[dataset][attribute][rep]):
        evaluation[dataset][attribute][f"hit_{rep}"] = 0.5
        evaluation[dataset][attribute]["total_hits"] += 0.5
      else:
        evaluation[dataset][attribute][f"hit_{rep}"] = 0

    evaluation[dataset][attribute]["average_hits"] = evaluation[dataset][attribute]["total_hits"] / n
    evaluation[dataset]["total_hits"] += evaluation[dataset][attribute]["average_hits"]
    print(f"Attribute {attribute}: average_hits = {evaluation[dataset][attribute]['average_hits']}")

  latex_dict["hits"] = evaluation[dataset]["total_hits"]
  latex_dict["failure"] = len(results_clean[dataset]) - evaluation[dataset]["total_hits"]
  latex_dict["accuracy"] = evaluation[dataset]["total_hits"] / len(results_clean[dataset])
  latex_dict_list.append(latex_dict)
  evaluation[dataset]["average_hits"] = evaluation[dataset]["total_hits"] / len(results_clean[dataset])
  print(f"Dataset average hits: {evaluation[dataset]['average_hits']}\n\n")
pd.DataFrame(latex_dict_list).to_csv("prompt1_outliers.csv")
  

 

 


Dataset: wine

Attribute Alcohol: average_hits = 0.0
Attribute Malic_acid: average_hits = 0.0
Attribute Ash: average_hits = 0.0
Attribute Alcalinity_of_ash: average_hits = 1.0
Attribute Magnesium: average_hits = 0.0
Attribute Total_phenols: average_hits = 0.0
Attribute Flavanoids: average_hits = 0.0
Attribute Nonflavanoid_phenols: average_hits = 0.0
Attribute Proanthocyanins: average_hits = 0.0
Attribute Color_intensity: average_hits = 0.0
Attribute Hue: average_hits = 0.0
Attribute OD280%2FOD315_of_diluted_wines: average_hits = 0.0
Attribute Proline: average_hits = 0.0
Dataset average hits: 0.07692307692307693


Dataset: ecoli

Attribute mcg: average_hits = 0.0
Attribute gvh: average_hits = 0.0
Attribute lip: average_hits = 0.0
Attribute chg: average_hits = 0.0
Attribute aac: average_hits = 0.0
Attribute alm1: average_hits = 0.0
Attribute alm2: average_hits = 0.0
Dataset average hits: 0.0


Dataset: glass

Attribute RI: average_hits = 0.0
Attribute Na: average_hits = 0.0
Attribute Mg: